# Callback

Макет описывает, как выглядит приложение, и представляет собой иерархическое дерево компонентов. Модуль `dash.html` предоставляет классы для всех тегов HTML. Модуль `dash.dcc` генерирует компоненты более высокого уровня, такие как элементы управления и графики.

Интерактивность приложения достигается за счет использования функций обратного вызова: функций, которые автоматически вызываются Dash всякий раз, когда изменяется свойство одного компонента, чтобы обновить какое-либо свойство в другом компоненте.

## Пример 1. Изменение содержимого блока на основе поля ввода

In [1]:
%%file 02_callbacks/input.py
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)

# создаем простой макет
app.layout = html.Div([
    html.H2("Измените значение в текстовом поле, чтобы увидеть обратные вызовы в действии!"),
    html.Div([
        dcc.Input(id='my-input', value='Начальное значение', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

])

# описываем функцию обратного вызова
# входные и выходные параметры функции описываются при помощи декоратора
# когда значение входного параметра меняется, задекорированная функция вызовется автоматически
# при запуске приложения все функции обратного вызова будут вызваны по одному разу
@app.callback(
    # изменяем значение children у компонента с ID my-output ...
    Output(component_id='my-output', component_property='children'),
    # ... на основе значения value у компонента c ID my-input
    Input(component_id='my-input', component_property='value') 
)
def update_output_div(input_value):
    return f'Output: {input_value}'

if __name__ == '__main__':
    app.run_server(debug=True)

Writing 02_callbacks/input.py


## Пример 2. Изменение визуализации

In [3]:
# для примера будем использовать этот датасет
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')
df.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [2]:
%%file 02_callbacks/graph_slider.py
import plotly.express as px
from dash import Dash, dcc, Input, Output, html
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')
app = Dash(__name__, external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div(
    [
        dcc.Graph(id='graph-with-slider'),
        dcc.Slider(
            df['year'].min(),
            df['year'].max(),
            step=None,
            value=df['year'].min(),
            marks={str(year): str(year) for year in df['year'].unique()},
            id='year-slider'
        ),
        html.H2('Горизонтальная ось:'),
        dcc.RadioItems(
            ['Linear', 'Log'],
            value='Linear',
            id='xaxis-type'
        )
    ]
)

# callback может принимать несколько входных параметров
@app.callback(
    # модифицируем фигуру на основе
    Output('graph-with-slider', 'figure'),
    # значения слайдера и селектора
    Input('year-slider', 'value'),
    Input('xaxis-type', 'value')
)
def _(selected_year, xaxis_type):
    # функция обратного вызова не должна модифицировать глобальные переменные
    filtered_df = df[df.year == selected_year]
    fig = px.scatter(
        filtered_df, 
        x="gdpPercap", 
        y="lifeExp",
        size="pop", 
        color="continent", 
        hover_name="country",
        log_x=(xaxis_type=='Log'), 
        size_max=55
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)

Writing 02_callbacks/graph_slider.py


## Пример 3. Цепочки обратных вызовов

In [3]:
%%file 02_callbacks/chain.py
import plotly.express as px
from dash import Dash, dcc, Input, Output, html
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')
app = Dash(__name__, external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div(
    [
        dcc.Graph(id='graph-with-slider'),
        html.H2("Континент:"),
        dcc.RadioItems(
            df['continent'].unique().tolist(),
            'Asia',
            id='continent-radio',
        ),
        html.H2("Страна:"),
        dcc.RadioItems(
            df[df['continent']=='Asia']['country'].unique().tolist(),
            value='Afghanistan',
            id='country-radio',
        ),
    ]
)

# изменение continent-radio меняет country-radio
@app.callback(
    Output('country-radio', 'options'),
    Output('country-radio', 'value'),
    Input('continent-radio', 'value')
)
def update_country_radio(continent_value):
    options = df[df['continent']==continent_value]['country'].unique()
    option = options[0]
    return options, option

# изменение country-radio и continent-radio меняет graph-with-slider
@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('continent-radio', 'value'),
    Input('country-radio', 'value')
)
def update_fig(continent, country):
    filtered_df = df[(df['continent']==continent) & (df['country']==country)]
    fig = px.scatter(
        filtered_df, 
        x="year", 
        y="lifeExp",
        size="pop", 
        size_max=55
    )
    return fig

if __name__ == "__main__":
    app.run_server(debug=True)

Writing 02_callbacks/chain.py


## Пример 4.

В некоторых ситуациях требуется сделать обратный вызов не по мере изменения некоторого поля, а после завершения ввода. 
В примерах выше любое изменение полей триггерит обратный вызов. 

`State` позволяет передать передавать дополнительные значения без запуска обратных вызовов.

In [9]:
%%file 02_callbacks/state.py
from dash import Dash, dcc, html, Input, Output, State

app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)

app.layout = html.Div(
    [
        dcc.Input(id='input-1-state', type='text', placeholder="Введите что-нибудь"),
        html.Br(),
        dcc.Input(id='input-2-state', type='text', placeholder="Введите что-нибудь"),
        html.Br(),
        html.Button(id='submit-button-state', n_clicks=0, children='Submit'),
        html.Br(),
        html.Div(id='output-state'),
        insta_div := html.Div()
    ]
)

@app.callback(
    Output(insta_div, 'children'),
    Input('input-1-state', 'value'), # изменение текста в этом поле триггерит обратный вызов
    Input('input-2-state', 'value') # изменение текста в этом поле триггерит обратный вызов
)
def _(input_1, input_2):
    return f'''
    Меняется моментально: 
    Значение поля 1: {input_1}.\n
    Значение поля 2: {input_2}.\n
    '''

@app.callback(
    Output('output-state', 'children'),
    Input('submit-button-state', 'n_clicks'), # свойство у объектов HTML
    State('input-1-state', 'value'), # изменение текста в этом поле не триггер обратный вызов
    State('input-2-state', 'value') # изменение текста в этом поле не триггер обратный вызов
)
def _(n_clicks, input_1, input_2):
    return f'''
    Меняется после нажатия 
    Кнопка была нажата {n_clicks} раз.\n
    Значение поля 1: {input_1}.\n
    Значение поля 2: {input_2}.\n
    '''

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 02_callbacks/state.py


Иногда для работы обратного вызова необходимо выполнить длительный подготовительный этап. Иногда результаты работы этого этапа можно использования для установки значений сразу нескольких элементов. 

Иногда удобнее, чтобы один обратный вызов подготовил данные, а другой - модифицировал состояние элементов. Это пример разделения переменных (состояния) между обратными вызовами.

Dash разрабатывался как stateless фреймворк. У такого подхода есть свои преимущества: возможность дешевого увеличения мощностей; отказоустойчивость при работе нескольких "копий" приложения.

Важно: сессия пользователя не равна процессу на сервере, т.е. каждый обратный вызов может быть вызван любым из доступных процессов. Именно поэтому нельзя модифицировать глобальные переменные, т.к. это может вызвать проблемы для другого пользователя. 

Чтобы безопасно разделить данные между несколькими процессами или серверами, нужно хранить данные в специальном месте, где они доступны всем процессам:
* в сессии пользователя (`dcc.Store`)
* на диске (файл, БД);
* в серверной памяти, разделенной между процессами (напр. Redis).

## Пример 5. Хранение результата в сессии пользователя

In [14]:
%%file 02_callbacks/store.py
from dash import Dash, dcc, html, Input, Output, State
import json
import numpy as np
import time
import plotly.express as px
import pandas as pd
def create_table(df):
    header = html.Thead(
        html.Tr([html.Th(col) for col in df.columns])
    )
    body = html.Tbody([
        html.Tr([
            html.Td(df.iloc[i][col]) for col in df.columns
        ]) for i in range(len(df))
    ])
    return header, body

app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)

app.layout = html.Div(
    [
        dcc.Dropdown(id='dropdown', options=["Normal", "Uniform"], value="Normal"),
        dcc.Graph(id='graph'),
        html.Table(id='table'),

        # dcc.Store для хранения промежуточных значений
        dcc.Store(id='intermediate-value')
    ]
)

# делаем препроцессинг и засовываем результат в Store
# в виде строки-JSON
@app.callback(
    Output('intermediate-value', 'data'),
    Input('dropdown', 'value')
)
def preprocessing(value):
    print("Sleeping...")
    time.sleep(3)
    if value == "Normal":
        data = np.random.random(size=(100, 2)).tolist()
    else:
        data = np.random.uniform(10, 20, size=(100, 2)).tolist()
    return json.dumps(data)

# после этого тригерятся остальные вызовы, основанные на Store
@app.callback(
    Output('graph', 'figure'),
    Input('intermediate-value', 'data')
)
def update_graph(data):
    data_df = pd.DataFrame(json.loads(data), columns=['x', 'y'])
    return px.scatter(data_df, x="x", y="y")

@app.callback(
    Output('table', 'children'),
    Input('intermediate-value', 'data')
)
def update_table(data):
    data_df = pd.DataFrame(json.loads(data), columns=['x', 'y'])
    return create_table(data_df)

if __name__ == '__main__':
    app.run_server(debug=True)    

Overwriting 02_callbacks/store.py


## Пример 6. Разделение данных и кэширование

In [16]:
from flask_caching import Cache

In [20]:
from dash import Dash, dcc, html, Input, Output, State
app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
server = app.server
CACHE_CONFIG = {
    # try 'FileSystemCache' if you don't want to setup redis
    'CACHE_TYPE': 'FileSystemCache',
    'CACHE_DIR': '02_callbacks/.'
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

In [22]:
%%file 02_callbacks/store_cache.py
from dash import Dash, dcc, html, Input, Output, State
import json
import numpy as np
import time
import plotly.express as px
import pandas as pd
from flask_caching import Cache

def create_table(df):
    header = html.Thead(
        html.Tr([html.Th(col) for col in df.columns])
    )
    body = html.Tbody([
        html.Tr([
            html.Td(df.iloc[i][col]) for col in df.columns
        ]) for i in range(len(df))
    ])
    return header, body

app = Dash(
    __name__,
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
# нужна дополнительная библиотека flask_caching
# создаем специальный объект для кэширования на диске
server = app.server
CACHE_CONFIG = {
    'CACHE_TYPE': 'FileSystemCache',
    'CACHE_DIR': '02_callbacks/.'
}
cache = Cache()
cache.init_app(app.server, config=CACHE_CONFIG)

app.layout = html.Div(
    [
        dcc.Dropdown(id='dropdown', options=["Normal", "Uniform"], value="Normal"),
        dcc.Graph(id='graph'),
        html.Table(id='table'),
        # dcc.Store для хранения промежуточных значений
        dcc.Store(id='intermediate-value')
    ]
)

# делаем препроцессинг и сохраняем результат в кэш
# который доступен всем процессам в любое время
@cache.memoize()
def global_store(value):
    print(f"Computing {value}...")
    time.sleep(3)
    if value == "Normal":
        data = np.random.random(size=(100, 2)).tolist()
    else:
        data = np.random.uniform(10, 20, size=(100, 2)).tolist()
    return pd.DataFrame(data, columns=['x', 'y'])

# тригерим вычисления и сохранение в кэш
# подаем сигнал, когда расчеты закончены
@app.callback(
    Output('intermediate-value', 'data'),
    Input('dropdown', 'value')
)
def preprocessing(value):
    global_store(value)
    return value

# после этого тригерятся остальные вызовы
# данные для визуализаций уже готовы после вызова preprocessing
# и лежат в global_store
@app.callback(
    Output('graph', 'figure'),
    Input('intermediate-value', 'data')
)
def update_graph(value):
    data_df = global_store(value)
    return px.scatter(data_df, x="x", y="y")

@app.callback(
    Output('table', 'children'),
    Input('intermediate-value', 'data')
)
def update_table(value):
    data_df = global_store(value)
    return create_table(data_df)

if __name__ == '__main__':
    app.run_server(debug=True)    

Overwriting 02_callbacks/store_cache.py


## Продвинутые функции:
* если возбудить исключение `PreventUpdate`, то выходные компоненты не будут обновляться;
* если нужно сохранить состояние только некоторых выходов, можно вернуть специальное значение `dash.no_update`
* внутри функции обратного вызова доступна переменная `dash.callback_context`, у которой есть поля:
    - `triggered` - список входов, которые триггерили контекст;
    - `inputs` и `states` - ID параметров функции
    - и еще некоторые другие поля и методы;
* чтобы не вызывать функции при старте приложения, можно передать дополнительный аргумент `prevent_initial_call` в декоратор
* функция обратного вызова может изменять компонент, который является ее входом (это можно использовать для синхронизации нескольких компонент)


## Обратные вызовы на клиентской стороне
Когда накладные расходы на обратный вызов становятся слишком большими и никакая другая оптимизация невозможна, обратный вызов может быть изменен для запуска непосредственно в браузере вместо отправки запроса в Dash

Особенности:
* обратные вызовы на клиентской стороне выполняются в главном потоке и блокируют выполнение;
* асинхронные обратные вызовы на клиентской стороне не поддерживаются;
* не подходят, если нужно работать с глобальными переменными или с БД

In [1]:
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import json
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')
df.head()

,country,year,pop,continent,lifeExp,gdpPercap
0,Afghanistan,1952,8425333.0,Asia,28.801,779.445314
1,Afghanistan,1957,9240934.0,Asia,30.332,820.853030
2,Afghanistan,1962,10267083.0,Asia,31.997,853.100710
3,Afghanistan,1967,11537966.0,Asia,34.020,836.197138
4,Afghanistan,1972,13079460.0,Asia,36.088,739.981106


In [7]:
%%file 02_callbacks/clientside.py
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import json
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

app = Dash(
    __name__, 
    external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css']
)
app.layout = html.Div(
    [
        dcc.Graph(id='clientside-graph'),
        # используем store для хранения данных для графиков
        dcc.Store(
            id='clientside-figure-store',
            data=[
                {
                    "x": df[df['country'] == 'Canada']['year'],
                    "y": df[df['country'] == 'Canada']['pop'],
                }
            ]
        ),
        "Индикатор",
        dcc.Dropdown(
            {
                'pop' : 'Population', 
                'lifeExp': 'Life Expectancy', 
                'gdpPercap': 'GDP per Capita'
            },
            'pop',
            id='clientside-graph-indicator'
        ),
        "Страна", 
        dcc.Dropdown(df["country"].unique(), 'Canada', id='clientside-graph-country'),
        "Масштаб осей",
        dcc.RadioItems(
            ['linear', 'log'],
            'linear',
            id='clientside-graph-scale'
        ),
        html.Hr(),
        html.Details(
            [
                html.Summary('Contents of figure storage'),
                dcc.Markdown(
                    id='clientside-figure-json'
                )
            ]
        )
    ]
)

# два обычных обратных вызова - для заполнения store ...
@app.callback(
    Output('clientside-figure-store', 'data'),
    Input('clientside-graph-indicator', 'value'),
    Input('clientside-graph-country', 'value')
)
def update_store_data(indicator, country):
    dff = df[df['country'] == country]
    return [{
        'x': dff['year'],
        'y': dff[indicator],
        'mode': 'markers'
    }]

# ... и для обновления тэга details
@app.callback(
    Output('clientside-figure-json', 'children'),
    Input('clientside-figure-store', 'data')
)
def generated_figure_json(data):
    return '```\n'+json.dumps(data, indent=2)+'\n```'

# функция обратного вызова на клиенской стороне
app.clientside_callback(
    # сама функция на JS в виде строки
    """
    function(data, scale) {
        return {
            'data': data,
            'layout': {
                 'yaxis': {'type': scale}
             }
        }
    }
    """,
    # обычные Input и Output
    Output('clientside-graph', 'figure'),
    Input('clientside-figure-store', 'data'),
    Input('clientside-graph-scale', 'value')
)

if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting 02_callbacks/clientside.py
